In [1]:
repo_directory = '/Users/iaincarmichael/Dropbox/Research/law/law-net/'

data_dir = '/Users/iaincarmichael/Data/courtlistener/'

import numpy as np
import sys
import matplotlib.pyplot as plt
from scipy.stats import rankdata
import cPickle as pickle

# graph package
import igraph as ig

# our code
sys.path.append(repo_directory + 'code/')
from setup_data_dir import setup_data_dir, make_subnetwork_directory
from pipeline.download_data import download_bulk_resource, download_master_edgelist, download_scdb
from helpful_functions import case_info

sys.path.append(repo_directory + 'vertex_metrics_experiment/code/')
from rankscore_experiment_sort import *
from rankscore_experiment_LR import *
from rankscore_experiment_match import *

from make_tr_edge_df import *


# which network to download data for
network_name = 'federal' # 'federal', 'ca1', etc


# some sub directories that get used
raw_dir = data_dir + 'raw/'
subnet_dir = data_dir + network_name + '/'
text_dir = subnet_dir + 'textfiles/'


# jupyter notebook settings
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
G = ig.Graph.Read_GraphML(subnet_dir + network_name +'_network.graphml')

## parameters from make snapshots

In [3]:
# vertex_metrics = ['indegree', 'outdegree', 'degree',
#                   'd_pagerank','u_pagerank',
#                   'authorities', 'hubs',
#                   #'d_eigen', 'u_eigen', # d_eigen is being problematic
#                   'u_eigen',
#                   'd_betweenness', 'u_betweenness',
#                   'd_closeness', 'u_closeness']

# # add recent citations
# vertex_metrics += ['recentcite_' + str(t) for t in np.arange(1, 10 + 1)]
# vertex_metrics += ['recentcite_' + str(t) for t in [15, 20, 25, 30, 35, 40]]

vertex_metrics = ['indegree', 'outdegree']

vertex_metrics += ['age', 'similarity']


vertex_metrics = ['indegree', 'outdegree']
active_years = range(1900, 2015 + 1)

## test parameters

In [4]:
test_seed = 4332,
num_test_cases = 10

test_cases = get_test_cases(G, active_years, num_test_cases, seed=test_seed)

# rank by sorting

In [5]:
%%time 
rankloss_sort = get_rankscores_sort(G, test_cases, vertex_metrics, subnet_dir)

CPU times: user 1min 13s, sys: 7.59 s, total: 1min 21s
Wall time: 1min 21s


In [6]:
MRS_sort = rankloss_sort['MRS'].mean().sort_values()
RR_sort = rankloss_sort['RR'].mean().sort_values()
PAK100_sort = rankloss_sort['PAK100'].mean().sort_values()
PAK1000_sort = rankloss_sort['PAK1000'].mean().sort_values()

MRS_sort

indegree     0.176123
outdegree    0.185956
dtype: float64

In [7]:
# histogram of scores

# plt.figure(figsize=[20, 20])
# k = 1
# h = ceil(scores_sort.shape[1] / 4.0)
# for c in sort_mean.index:
#     plt.subplot(h, 4, k)
#     plt.hist(scores_sort[c])
#     plt.xlabel(c)
    
#     k += 1

# Match

In [8]:
num_to_keep = 5000

In [9]:
%%time
rankloss_match = get_rankscores_match(G, test_cases, vertex_metrics, subnet_dir, num_to_keep)

CPU times: user 6min 15s, sys: 2min 3s, total: 8min 18s
Wall time: 9min 34s


In [10]:
MRS_match = rankloss_match['MRS'].mean().sort_values()
RR_match = rankloss_match['RR'].mean().sort_values()
PAK100_match = rankloss_match['PAK100'].mean().sort_values()
PAK1000_match = rankloss_match['PAK1000'].mean().sort_values()

MRS_match

indegree     0.237019
outdegree    0.280063
dtype: float64

# logistic regression

## make training data for logistic regression

In [11]:
# how many abset edges to add
num_absent_edges = len(G.es)
seed_edge_df = 32432

# how to normalize yearly metrics
metric_normalization = 'mean'

In [30]:
%%time 

# make_tr_edge_df(G, subnet_dir,
#                 active_years, num_absent_edges,
#                 vertex_metrics, metric_normalization,
#                 seed=seed_edge_df)

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 7.87 µs


## rank by logistic regression

In [31]:
%%time
rankloss_LR, LogRegs = get_rankscores_LR(G, test_cases, vertex_metrics, subnet_dir,
                                         metric_normalization)              

CPU times: user 4min 52s, sys: 35.1 s, total: 5min 27s
Wall time: 5min 27s


In [61]:
MRS_LR = rankloss_LR['MRS'].mean().sort_values()
RR_LR = rankloss_LR['RR'].mean().sort_values()
PAK100_LR = rankloss_LR['PAK100'].mean().sort_values()
PAK1000_LR = rankloss_LR['PAK1000'].mean().sort_values()

MRS_LR

u_betweenness    0.320712
d_betweenness    0.389026
u_closeness      0.580034
d_closeness      0.583389
u_pagerank       0.588722
d_pagerank       0.630018
authorities      0.831034
u_eigen          0.850612
hubs             0.858392
degree           0.889295
recentcite_20    0.894102
recentcite_15    0.896261
recentcite_30    0.896953
recentcite_25    0.897073
outdegree        0.898077
recentcite_35    0.899801
recentcite_40    0.903414
indegree         0.912617
recentcite_10    0.915899
recentcite_9     0.921747
recentcite_8     0.928158
recentcite_7     0.934804
recentcite_6     0.943219
recentcite_1     0.944017
recentcite_5     0.952080
similarity       0.954872
recentcite_2     0.956526
recentcite_4     0.957748
age              0.959504
recentcite_3     0.959609
dtype: float64

# cache results

In [34]:
with open(subnet_dir + 'results/rankloss_sort.p', 'wb') as fp:
    pickle.dump(rankloss_sort, fp)
    
with open(subnet_dir + 'results/rankloss_match.p', 'wb') as fp:
    pickle.dump(rankloss_match, fp)

with open(subnet_dir + 'results/rankloss_LR.p', 'wb') as fp:
    pickle.dump(rankloss_LR, fp)
    
with open(subnet_dir + 'results/LogRegs.p', 'wb') as fp:
    pickle.dump(LogRegs, fp)

In [36]:
rankloss_sort = pickle.load( open( subnet_dir + 'results/rankloss_sort.p', "rb" ) )
rankloss_match = pickle.load( open( subnet_dir + 'results/rankloss_match.p', "rb" ) )
rankloss_LR = pickle.load( open( subnet_dir + 'results/rankloss_LR.p', "rb" ) )
LogRegs = pickle.load( open( subnet_dir + 'results/LogRegs.p', "rb" ) )

In [24]:
# scores_sort.to_csv(subnet_dir + 'results/scores_sort.csv', index=True)
# scores_search.to_csv(subnet_dir + 'results/scores_search.csv', index=True)
# scores_LR.to_csv(subnet_dir + 'results/scores_LR.csv', index=True)
# scores_LR_logloss.to_csv(subnet_dir + 'results/scores_LR_logloss.csv', index=True)

# with open(subnet_dir + 'results/LogRegs.p', 'wb') as fp:
#     pickle.dump(LogRegs, fp)


# results

In [62]:
df_metric = pd.DataFrame(columns=['sort', 'match', 'LR'],
                         index = range(len(vertex_metrics)))

df_metric['sort'] = MRS_sort.index
df_metric['match'] = MRS_match.index
df_metric['LR'] = MRS_LR.index

df_metric

In [69]:
rankscores = pd.DataFrame(columns=['sort', 'match', 'LR'],
                         index = vertex_metrics)

rankscores['sort'] = MRS_sort
rankscores['match'] = MRS_match
rankscores['LR'] = MRS_LR

In [72]:
rankscores.sort_values(by='sort', ascending=True)

,sort,match,LR
similarity,0.045186,0.087850,0.954872
age,0.203464,0.272298,0.959504
recentcite_7,0.209990,0.288943,0.934804
recentcite_6,0.210109,0.285066,0.943219
recentcite_8,0.210439,0.290901,0.928158
recentcite_5,0.213641,0.286009,0.952080
recentcite_9,0.213744,0.297070,0.921747
u_eigen,0.214163,0.336644,0.850612
recentcite_10,0.214385,0.301320,0.915899
recentcite_4,0.223274,0.291334,0.957748


In [73]:
rs_ranking = rankscores.apply(lambda c: rankdata(c))

In [74]:
rs_ranking.sort_values(by='sort')

,sort,match,LR
similarity,1.0,1.0,26.0
age,2.0,2.0,29.0
recentcite_7,3.0,5.0,22.0
recentcite_6,4.0,3.0,23.0
recentcite_8,5.0,6.0,21.0
recentcite_5,6.0,4.0,25.0
recentcite_9,7.0,9.0,20.0
u_eigen,8.0,15.0,8.0
recentcite_10,9.0,10.0,19.0
recentcite_4,10.0,7.0,28.0
